# Lab 2: Linear Regression

## Theoretical Overview

$$ H(x) = Wx + b $$

$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$

 - $H(x)$: 주어진 $x$ 값에 대해 예측을 어떻게 할 것인가
 - $cost(W, b)$: $H(x)$ 가 $y$ 를 얼마나 잘 예측했는가

## Imports

In [0]:
import torch
import torch.optim as optim   #다양한 최적화 알고리즘을 구현해놓은 패키지

In [0]:
# For reproducibility
# seed 함수를 호출해서 같은 seed 값으로 고정시켰다.
# seed는 랜덤하게 숫자를 생성할 때, 동일한 값이 호출되도록 한다.

torch.manual_seed(1)

## Data

We will use fake data for this example.

> 들여쓴 블록



In [0]:
# 선형회귀를 위한 (x1,y1)=(1,1), (x2,y2)=(2,2), (x3,y3)=(3,3)
# FloatTensor를 이용해 값을 할당했다.

x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

In [0]:
print(x_train)
print(x_train.shape)


tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


In [0]:
print(y_train)
print(y_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


기본적으로 PyTorch는 NCHW 형태이다.

## Weight Initialization

In [0]:
 # zeros() 0으로 초기화. 

W = torch.zeros(1, requires_grad=True)   # 사이즈는 1, requires_grad=True 학습을 통해서 W값을 찾겠다.
print(W)                                 # W는 0으로 초기화 되고, 학습용 변수이다.

tensor([0.], requires_grad=True)


In [0]:
b = torch.zeros(1, requires_grad=True)
print(b)

tensor([0.], requires_grad=True)


## Hypothesis

- 해결하려는 문제의 모델을 Linear 선형방정식으로 정한다. 

$$ H(x) = Wx + b $$

In [0]:
# 가설설정 H(x)

hypothesis = x_train * W + b
print(hypothesis)     # 모두 0 으로 초기화되어있기때문에, 0 0 0 으로 출력됌.

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


## Cost

- cost함수 정의하기
- cost함수는 예측된 값(가설)과 정답의 차이를 계산한다.
- 예측된 값=hypothesis = x_train*W+b
- 정답 = y_train

$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$

In [0]:
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


In [0]:
print(y_train)

tensor([[1.],
        [2.],
        [3.]])


In [0]:
print(hypothesis - y_train)

tensor([[-1.],
        [-2.],
        [-3.]], grad_fn=<SubBackward0>)


In [0]:
print((hypothesis - y_train) ** 2)    # hypothesis = H(x)

tensor([[1.],
        [4.],
        [9.]], grad_fn=<PowBackward0>)


In [0]:
# cost function을 정의

cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor(4.6667, grad_fn=<MeanBackward0>)


## Gradient Descent

- cost함수를 최소화하는 W, b 값을 구하기 위해 Gradient Descent를 이용
- Gradient Descent 알고리즘을 직접 구현할 수 있지만 
- 파이토치에 이미 구현되어있는 **optimizer**를 활용할 수 있다.

In [0]:
# W := W - alpha*d/dw*cost(w)
# 알파값은 learning rate  => lr = 0.01

# SGD 알고리즘 Gradient Descent

optimizer = optim.SGD([W, b], lr=0.01)    #optimizer로 구할 변수는 W와 b이다 라고 지정.

In [0]:
# 옵티마이저(gradient값을) 0으로 초기화
optimizer.zero_grad()

# cost계산 !!!  미분값 계산해서
cost.backward()

# 옵티마이저 갱신
optimizer.step()

In [0]:
print(W)    # W, b가 초기값 0에서 각각 갱신되었다.
print(b)

tensor([0.0933], requires_grad=True)
tensor([0.0400], requires_grad=True)


갱신된 w와 b를 넣어 가설을 설정하고 확인하기

In [0]:
# 갱신된 W와 b를 넣어 hypothesis(가설) 을 다시 설정하고, 
# 아래에서 cost function을 계산하여 갱신시키기

hypothesis = x_train * W + b
print(hypothesis)             #y1, y2, y3 의 값이 출력 

tensor([[0.1333],
        [0.2267],
        [0.3200]], grad_fn=<AddBackward0>)


cost함수에 업데이트된 가설을 넣어 계산해보기

In [0]:
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)     # cost도 4.6667 -> 3.6927로 갱신되었다.

# 이를 계속 반복적으로 하다보면 cost값이 계속 작아질 것이다.
# 결국 더이상 cost값이 변하지 않게 될 것이다. 이때 학습을 멈춘다.

tensor(4.6667, grad_fn=<MeanBackward0>)


## Training with Full Code

In reality, we will be training on the dataset for multiple epochs. This can be done simply with loops.

In [0]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):    # 1000번 반복
    
    # H(x) 계산 (갱신된 W,b를 이용하여 가설 설정)
    hypothesis = x_train * W + b
    
    # cost 계산  
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선 ( W,b 갱신)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))


# 이를 반복하다보면 cost는 작아지고, W는 1에 수렴하게 된다.

# W: 0.985, b: 0.033 Cost: 0.000158

Epoch    0/1000 W: 0.093, b: 0.040 Cost: 4.666667
Epoch  100/1000 W: 0.873, b: 0.289 Cost: 0.012043
Epoch  200/1000 W: 0.900, b: 0.227 Cost: 0.007442
Epoch  300/1000 W: 0.921, b: 0.179 Cost: 0.004598
Epoch  400/1000 W: 0.938, b: 0.140 Cost: 0.002842
Epoch  500/1000 W: 0.951, b: 0.110 Cost: 0.001756
Epoch  600/1000 W: 0.962, b: 0.087 Cost: 0.001085
Epoch  700/1000 W: 0.970, b: 0.068 Cost: 0.000670
Epoch  800/1000 W: 0.976, b: 0.054 Cost: 0.000414
Epoch  900/1000 W: 0.981, b: 0.042 Cost: 0.000256
Epoch 1000/1000 W: 0.985, b: 0.033 Cost: 0.000158


값을 예측해보기. 구해진 w와 b를 통해.
- W: 0.985, b: 0.033 Cost: 0.000158

In [0]:
# 가설설정 = 예측하고자 하는 값 

# H(x) = Wx + b
# 시험성적 = 1 * 7 + 0.033 = 